In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE146383"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE146383"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE146383.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE146383.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE146383.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Effect of Age on Severity and Recovery of Acute Multiple Sclerosis Attack"
!Series_summary	"Pediatric MS patients suffer from severe first and second relapse but better recovery explained by difference in age-restricted transcriptional profiles associated with antigen-presentation and B-cell activation"
!Series_summary	"Herein, we compared the blood mononuclear cell transcriptome of pediatric and adult MS patients with recovery (PDMS-rec, ADMS-rec) and without recovery (PDMS-norec, ADMS-norec) 6 months after relapse. Healthy pediatric and adult subjects (PDC, ADC) were used as controls."
!Series_overall_design	"A total of 30 MS patients (14 PDMS-rec and ADMS-rec, 16 PDMS-norec and ADMS-norec , 55 PDC andADC) that met the inclusion criteria were analyzed."
Sample Characteristics Dictionary:
{0: ['gender: Female', 'gender: Male'], 1: ['age (years): 16', 'age (years): 38', 'age (years): 17', 'age (years): 37', 'age (years): 14', 'age (years): 15', 'a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the title and summary, this dataset involves transcriptome data, which implies gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary, we can see:
# The trait information isn't directly available in the sample characteristics
trait_row = None  # Not directly available in the sample characteristics
age_row = 1       # Age information is available at index 1
gender_row = 0    # Gender information is available at index 0

# 2.2 Data Type Conversion
# For trait: We define this function even though we don't have trait data
def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait values to binary format.
    Based on the summary, we have MS patients and healthy controls.
    """
    if not value or ':' not in value:
        return None
    value = value.lower().split(':', 1)[1].strip()
    
    # Based on the background info, there are MS patients and healthy controls
    if 'ms' in value or 'multiple sclerosis' in value:
        return 1  # MS patient
    elif 'control' in value or 'healthy' in value:
        return 0  # Healthy control
    else:
        return None  # Unknown

# For age: Convert to continuous values
def convert_age(value: str) -> Optional[float]:
    """Convert age values to continuous format."""
    if not value or ':' not in value:
        return None
    try:
        # Extract the value after the colon and convert to float
        age = float(value.split(':', 1)[1].strip())
        return age
    except (ValueError, TypeError):
        return None

# For gender: Convert to binary (0 for female, 1 for male)
def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if not value or ':' not in value:
        return None
    value = value.lower().split(':', 1)[1].strip()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we still extract age and gender information if available
if clinical_file := os.path.join(in_cohort_dir, "clinical_data.csv"):
    if os.path.exists(clinical_file):
        clinical_data = pd.read_csv(clinical_file)
        
        # Extract available clinical features (age and gender)
        selected_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,  # This is None
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(selected_features)
        print("Preview of extracted clinical features:", preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_features.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [4]:
# 1. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 3. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene data dimensions: 22215 genes × 85 samples


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers shown in the output
# These appear to be Affymetrix probe IDs (like '1007_s_at', '1053_at', etc.)
# which are not standard human gene symbols and will need to be mapped

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers in both gene expression and annotation data
# From previous outputs we can see:
# - Gene expression data row indices are identifiers like '1007_s_at', '1053_at'
# - Gene annotation has 'ID' column with same format, and 'Gene Symbol' column with gene symbols

# 2. Extract the gene identifier and gene symbol columns
# We need to map from 'ID' column to 'Gene Symbol' column
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a sample of the mapping to verify
print("\nSample of gene mapping (ID to Gene Symbol):")
print(gene_mapping.head())
print(f"Mapping shape: {gene_mapping.shape}")

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Print info about the resulting gene data
print("\nAfter mapping to gene symbols:")
print(f"Gene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First 10 gene symbols:")
print(gene_data.index[:10])



Sample of gene mapping (ID to Gene Symbol):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Mapping shape: (21225, 2)

After mapping to gene symbols:
Gene data dimensions: 13830 genes × 85 samples
First 10 gene symbols:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since trait_row was None in Step 2, we need to handle missing trait data
# Create a clinical dataframe with age and gender information from the original clinical_data
# obtained in step 1, rather than trying to load from a file that doesn't exist
clinical_features = pd.DataFrame()

# Extract age data if available
if age_row is not None:
    age_data = get_feature_data(clinical_data, age_row, 'Age', convert_age)
    clinical_features = pd.concat([clinical_features, age_data])

# Extract gender data if available
if gender_row is not None:
    gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
    clinical_features = pd.concat([clinical_features, gender_data])

# Transpose to have samples as rows and features as columns
clinical_features = clinical_features.T

print(f"Clinical features shape: {clinical_features.shape}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
# Check if the sample IDs need alignment
gene_samples = set(normalized_gene_data.columns)
clinical_samples = set(clinical_features.index)
common_samples = gene_samples.intersection(clinical_samples)

print(f"Gene data samples: {len(gene_samples)}")
print(f"Clinical data samples: {len(clinical_samples)}")
print(f"Common samples: {len(common_samples)}")

# If no common samples, we need to align the sample IDs
if len(common_samples) == 0:
    print("No common sample IDs found, attempting to standardize IDs...")
    
    # Create a mapping from gene_data column names to clinical_features index
    # This is specific to GEO datasets where sample IDs follow a pattern
    # First, convert both to strings to ensure consistent comparison
    gene_samples_str = [str(s) for s in normalized_gene_data.columns]
    clinical_samples_str = [str(s) for s in clinical_features.index]
    
    # For GEO data, both should contain GSM identifiers
    # Try to match based on partial string matching
    id_map = {}
    for g_id in gene_samples_str:
        for c_id in clinical_samples_str:
            if g_id in c_id or c_id in g_id:
                id_map[g_id] = c_id
                break
    
    if id_map:
        print(f"Found {len(id_map)} sample ID mappings")
        # Rename columns in gene_data to match clinical_features
        gene_data_aligned = normalized_gene_data.copy()
        gene_data_aligned.columns = [id_map.get(str(col), col) for col in gene_data_aligned.columns]
        
        # Now link the data
        linked_data = pd.concat([clinical_features, gene_data_aligned.T], axis=1, join='inner')
    else:
        print("Could not align sample IDs, using original IDs")
        # Since we can't align, just try direct concatenation
        linked_data = pd.concat([clinical_features, normalized_gene_data.T], axis=1)
else:
    # If there are common samples, use only those
    linked_data = pd.concat([clinical_features.loc[list(common_samples)], 
                             normalized_gene_data.T.loc[list(common_samples)]], 
                            axis=1)

print(f"Linked data shape: {linked_data.shape}")

# 4. Since no trait data is available, we call validate_and_save_cohort_info with is_final=False
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False  # No trait data
)

print(f"Dataset marked as not usable due to missing trait information.")

Gene data shape after normalization: (13542, 85)
First 5 gene symbols after normalization: Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Multiple_sclerosis/gene_data/GSE146383.csv
Clinical features shape: (85, 2)
Clinical features preview:
{'Age': [16.0, 38.0, 17.0, 37.0, 14.0], 'Gender': [0.0, 0.0, 1.0, 0.0, 0.0]}
Clinical data saved to ../../output/preprocess/Multiple_sclerosis/clinical_data/GSE146383.csv
Gene data samples: 85
Clinical data samples: 85
Common samples: 85
Linked data shape: (85, 13544)
Dataset marked as not usable due to missing trait information.
